## Data clean

In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from langdetect import detect
import emoji

In [4]:
df = pd.read_csv("tiktok_google_play_reviews.csv")
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOHRz-11c0apHLSKHHp52FxUXsQS9Z88wP3sWc5...,MR LOL GAMER,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,5,0,23.8.4,2022-04-05 23:18:30,NaN,NaN
1,gp:AOqpTOF6mFDEkIypmyT3shDLjPHg8zB3kdns2W36ahp...,Dino Kljako,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome app! Too many people on it where it's ...,5,0,NaN,2022-04-05 23:18:21,NaN,NaN
2,gp:AOqpTOGtqU4sb8vuVo3-eB7kIXWoBn-0YCUZ1SnPRKS...,Olivia Harding,https://play-lh.googleusercontent.com/a/AATXAJ...,Not bad,5,0,23.9.5,2022-04-05 23:17:34,NaN,NaN
3,gp:AOqpTOFHDm-Qa5R6jCpOGTFT2qr1_PKbCTbBNPahCEn...,Keli We,https://play-lh.googleusercontent.com/a-/AOh14...,It is good,2,0,22.2.5,2022-04-05 23:17:04,NaN,NaN
4,gp:AOqpTOFB6Ndao8IHRpOJRmbSknwMGxHcwYzux93YyXI...,Mavis Kotoka,https://play-lh.googleusercontent.com/a/AATXAJ...,Very interesting app,5,0,22.1.5,2022-04-05 23:17:04,NaN,NaN


In [5]:
df[['content', 'score', 'thumbsUpCount']].head()

,content,score,thumbsUpCount
0,Good,5,0
1,Awesome app! Too many people on it where it's ...,5,0
2,Not bad,5,0
3,It is good,2,0
4,Very interesting app,5,0


### Data clean---thumbupcount remove 0 or 1

In [6]:
# Removing rows where thumbsUpCount is 0 or 1
df_filtered = df[df['thumbsUpCount'] > 1]

# Display the shape of the filtered DataFrame to see how many rows are left
df_filtered.shape

(7117, 10)

### NLP

In [7]:
# Define a function to analyze the sentiment of comments
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

# Apply the function to the filtered comments
df_filtered['sentiment'], df_filtered['subjectivity'] = zip(*df_filtered['content'].astype(str).apply(analyze_sentiment))

# Display the first few rows of the DataFrame with sentiment analysis
df_filtered[['content', 'sentiment', 'subjectivity']].head(20)

C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\2303948014.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['sentiment'], df_filtered['subjectivity'] = zip(*df_filtered['content'].astype(str).apply(analyze_sentiment))
C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\2303948014.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['sentiment'], df_filtered['subjectivity'] = zip(*df_filtered['content'].astype(str).apply(analyze_sentiment))


,content,sentiment,subjectivity
20,Just did recent update 4/3/22 . Videos stoppin...,0.000000,0.250000
214,thanks the app is good and a lot of the crashe...,0.450000,0.400000
640,It is a good app to be entertained and it is f...,0.250000,0.575000
705,You get a one-star until you fix this. Voice t...,-0.100000,0.266667
876,It's so fun,0.300000,0.200000
1102,My go to app for entertainment,0.000000,0.000000
1366,لا استطيع تفعيل ميزة هداية الفيديو في تركيا .ع...,0.000000,0.000000
1950,🇵🇰,0.000000,0.000000
2010,This app is best but as videos are repeated ti...,0.250000,0.361905
2145,"I like it i love it, BUT, Tik Tok is racist,Le...",0.035017,0.281713


### Figure 3 in paper

In [8]:
# sentiment > 0.5
positive_sentiments = df_filtered[df_filtered['sentiment'] > 0.5]

# sentiment <= -0.2
negative_sentiments = df_filtered[df_filtered['sentiment'] <= -0.2]

positive_sentiments[['content', 'sentiment', 'subjectivity']].head(20)

,content,sentiment,subjectivity
3174,This is too good 😘apps,0.700000,0.600000
3795,Dud they keep banning me for NO REASON AT ALL....,0.600000,0.900000
4039,Love this app Very good app Love 💕 tik tok ❤️❤...,0.602500,0.645000
4057,Good for watching videos in free time.,0.550000,0.700000
4858,Tiktok is best social media app👌💞💞,0.516667,0.183333
8355,I like tiktok App ⭐⭐⭐ ⭐⭐ Rate this App 📧 good ...,0.700000,0.600000
8444,Do i have to explain this app is just tha best 👌,1.000000,0.300000
8961,I love tik tok! Its amazing!!!,0.812500,0.750000
8983,It's the best app,1.000000,0.300000
9108,I am very happy. Works in your apps. As far as...,0.670000,0.926667


### Figure 4 in paper

In [9]:
negative_sentiments[['content', 'sentiment', 'subjectivity']].head(20)

,content,sentiment,subjectivity
2513,Not a good alarm clock don't install,-0.350000,0.600000
4955,My tik tok hasnt been working for the past few...,-0.215814,0.254924
5245,All I am seeing on this application lately is ...,-0.325000,0.400000
10207,very bad app don't use it they are trash,-0.910000,0.866667
10916,Stupid app and dead support team no-one answer...,-0.500000,0.700000
10964,I had to update my phone number and I didn't h...,-0.290043,0.531602
11049,It's really bad whenever I report a problem th...,-0.375000,0.558333
11630,I dont like the new user interface. They keep ...,-0.221212,0.484848
12041,April 2 2022 terrible running on the backgroun...,-1.000000,1.000000
12965,THIS IS ANNOYING. MY FRIEND JOIN MY ACCOUNT AN...,-1.000000,0.900000


### Remove non english comment since NLP not work for them

In [10]:
# Function to detect the language of a text
def detect_language(text):
    try:
        return detect(text)
    except:
        return None  # return None if language detection fails

# Apply the function to the content column
df_filtered['language'] = df_filtered['content'].apply(detect_language)

# Filter the DataFrame to keep only English comments
df_english = df_filtered[df_filtered['language'] == 'en']

C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\2786171354.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['language'] = df_filtered['content'].apply(detect_language)


### Word acount variable created

In [11]:
df_english['word_count'] = df_english['content'].str.split().str.len()

df_english[['content', 'word_count']].head()

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, float('inf')]

labels = ['1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101+']

df_english['word_category'] = pd.cut(df_english['word_count'], bins=bins, labels=labels, right=False)

df_english[['content', 'word_count', 'word_category']].head()

C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\90214824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['word_count'] = df_english['content'].str.split().str.len()
C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\90214824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['word_category'] = pd.cut(df_english['word_count'], bins=bins, labels=labels, right=False)


,content,word_count,word_category
20,Just did recent update 4/3/22 . Videos stoppin...,29,21-30
214,thanks the app is good and a lot of the crashe...,15,11-20
640,It is a good app to be entertained and it is f...,40,41-50
705,You get a one-star until you fix this. Voice t...,38,31-40
876,It's so fun,3,1-10


### Emoji dummy variable created

In [12]:
# Function to check if a string contains any emoji
def contains_emoji(content):
    # Converting all emojis to text strings
    text = emoji.demojize(content)
    # If the converted text is different from the original, there were emojis
    return 1 if text != content else 0

# Applying the function to the 'content' column
df_english['contains_emoji'] = df_english['content'].apply(contains_emoji)

C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\2856684241.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['contains_emoji'] = df_english['content'].apply(contains_emoji)


### Create dummy variable for sentiment and replay. log the thumbsupcount

In [13]:
# Calculating the median of sentiment scores
sentiment_median = df_english['sentiment'].median()

# Creating a dummy variable for sentiment
# If sentiment is greater than or equal to the median, it's 1, else 0
df_english['sentiment_dummy'] = np.where(df_english['sentiment'] >= sentiment_median, 1, 0)

# Creating a dummy variable for reply content
# Check if the reply content is not NaN and not an empty string
df_english['reply_dummy'] = df_english['replyContent'].apply(lambda x: 1 if pd.notna(x) and len(str(x).strip()) > 0 else 0)

# Log the thumbsUpCount
df_english['log_thumbsUpCount'] = np.log(df_english['thumbsUpCount'])

C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\1549840301.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['sentiment_dummy'] = np.where(df_english['sentiment'] >= sentiment_median, 1, 0)
C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\1549840301.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english['reply_dummy'] = df_english['replyContent'].apply(lambda x: 1 if pd.notna(x) and len(str(x).strip()) > 0 else 0)
C:\Users\yuhai\AppData\Local\Temp\ipykernel_35708\1549840301.py:13: S

In [14]:
df_english.to_csv('processed_tiktok_reviews_updated.csv', index=False)